In [10]:
#!/usr/bin/env python
import pika
import psycopg2
import json

credentials = pika.PlainCredentials('guest', 'guest')
connection = pika.ConnectionParameters('rabbitmq',
                                       5672,
                                       '/',
                                       credentials)

connection = pika.BlockingConnection(connection)
channel = connection.channel()

channel.queue_declare(queue='genetico')

conn = psycopg2.connect(host="postgres",
                        database="genetico", 
                        user="postgres", 
                        password="Postgres2019!")

cur = conn.cursor()

def create_table(name):
    create_table_query = '''CREATE TABLE IF NOT EXISTS {}
      (RUN            TEXT    NOT NULL,
       GENERATION      INT    NOT NULL,
       VARIABLES       TEXT    NOT NULL,
       FITNESS       NUMERIC(12,6)    NOT NULL); '''.format(name)
    cur.execute(create_table_query)
    conn.commit()
    
    
def callback(ch, method, properties, text):
    text= json.loads(str(text, 'utf-8'))
    sql = """INSERT INTO {}(run, generation, variables, fitness)
             VALUES(%s, %s, %s, %s);""".format(text['experiment'])
    create_table(text['experiment'])
    cur.execute(sql, (text['run'], text['generation'], text['variables'], text['fitness'],))
    conn.commit()
    
channel.basic_consume(queue='genetico',
                      auto_ack=True,
                      on_message_callback=callback)

'ctag1.4478ff820b70479abe89a2a8b075ad23'

In [ ]:
print(' [*] Waiting for messages. To exit press CTRL+C')
channel.start_consuming()

 [*] Waiting for messages. To exit press CTRL+C
